<a href="https://www.kaggle.com/code/bijaybeezoe/excelerate-assignment1?scriptVersionId=286306362" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

path = "/kaggle/input/excelerate/SLU Opportunity Wise Data-1710158595043.xlsx"

df = pd.read_excel(path)

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nDtypes:")
print(df.dtypes)

print("\nFirst 5 rows:")
print(df.head().to_string(index=False))


Shape: (8558, 16)

Columns:
['Learner SignUp DateTime', 'Opportunity Id', 'Opportunity Name', 'Opportunity Category', 'Opportunity End Date', 'First Name', 'Date of Birth', 'Gender', 'Country', 'Institution Name', 'Current/Intended Major', 'Entry created at', 'Status Description', 'Status Code', 'Apply Date', 'Opportunity Start Date']

Dtypes:
Learner SignUp DateTime            object
Opportunity Id                     object
Opportunity Name                   object
Opportunity Category               object
Opportunity End Date               object
First Name                         object
Date of Birth                      object
Gender                             object
Country                            object
Institution Name                   object
Current/Intended Major             object
Entry created at           datetime64[ns]
Status Description                 object
Status Code                         int64
Apply Date                         object
Opportunity Start Date  

In [2]:
date_cols = [
    "Learner SignUp DateTime",
    "Opportunity End Date",
    "Date of Birth",
    "Apply Date",
    "Opportunity Start Date",
]

for col in date_cols:
    if col in df.columns:
        # Try multiple formats by letting pandas infer
        df[col] = pd.to_datetime(df[col], errors="coerce")
        print(f"Parsed '{col}' to datetime; nulls after parse: {df[col].isna().sum()}")

print("\nDtypes AFTER date parsing:")
print(df.dtypes)

print("\nSample 5 rows AFTER date parsing:")
print(df.head().to_string(index=False))


Parsed 'Learner SignUp DateTime' to datetime; nulls after parse: 295
Parsed 'Opportunity End Date' to datetime; nulls after parse: 1262
Parsed 'Date of Birth' to datetime; nulls after parse: 0
Parsed 'Apply Date' to datetime; nulls after parse: 307
Parsed 'Opportunity Start Date' to datetime; nulls after parse: 4637

Dtypes AFTER date parsing:
Learner SignUp DateTime    datetime64[ns]
Opportunity Id                     object
Opportunity Name                   object
Opportunity Category               object
Opportunity End Date       datetime64[ns]
First Name                         object
Date of Birth              datetime64[ns]
Gender                             object
Country                            object
Institution Name                   object
Current/Intended Major             object
Entry created at           datetime64[ns]
Status Description                 object
Status Code                         int64
Apply Date                 datetime64[ns]
Opportunity Start Date  

In [3]:
def get_categorical_columns(df: pd.DataFrame):
    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
    print(f"Categorical-like columns ({len(cat_cols)}):")
    for c in cat_cols:
        print(" -", c)
    return cat_cols

def show_categorical_value_counts(
    df: pd.DataFrame,
    cols=None,
    top_n=None,
    sort_labels=True
):
    if cols is None:
        cols = get_categorical_columns(df)

    for col in cols:
        print("\n" + "="*80)
        print(f"Column: {col}")
        s = df[col].astype(str).str.strip()
        vc = s.value_counts(dropna=False)
        if sort_labels:
            vc = vc.sort_index()  
        if top_n is not None:
            vc = vc.head(top_n)
        print(vc.to_string())

cat_cols = get_categorical_columns(df)
print("\n--- Value counts for a few key columns (Gender, Country, Institution Name) ---")
show_categorical_value_counts(df, cols=["Gender", "Country", "Institution Name"], top_n=None, sort_labels=True)


Categorical-like columns (9):
 - Opportunity Id
 - Opportunity Name
 - Opportunity Category
 - First Name
 - Gender
 - Country
 - Institution Name
 - Current/Intended Major
 - Status Description

--- Value counts for a few key columns (Gender, Country, Institution Name) ---

Column: Gender
Gender
Don't want to specify      15
Female                   3522
Male                     5018
Other                       3

Column: Country
Country
Afghanistan                                  1
Algeria                                      1
American Samoa                               4
Australia                                    4
Azerbaijan                                   1
Bangladesh                                  65
Belarus                                      3
Bhutan                                       1
Botswana                                     2
Brazil                                       2
British Indian Ocean Territory               3
Cameroon                                

In [4]:
from collections import Counter
import re

norm_to_canonical = {
    # Saint Louis University
    "saint louis university": "Saint Louis University",
    "saint louis univeristy": "Saint Louis University",
    "saint louis univerisity": "Saint Louis University",
    "saint louis univeristy": "Saint Louis University",
    "saint louis unveracity": "Saint Louis University",
    "saint louis uinversity": "Saint Louis University",
    "saint lious university": "Saint Louis University",
    "saintlouis university": "Saint Louis University",
    "saintlouis": "Saint Louis University",
    "saint louis": "Saint Louis University",
    "saint louis university": "Saint Louis University",
    "saint louis university mo": "Saint Louis University",
    "saint louis university-main campus": "Saint Louis University",
    "saint louis university i": "Saint Louis University",
    "saint louis university madrid": "Saint Louis University",
    "saint louis university-madrid": "Saint Louis University",
    "saint louis university-madrid": "Saint Louis University",
    "saint louise university": "Saint Louis University",
    "saint lousis university": "Saint Louis University",
    "saintlouis university": "Saint Louis University",
    "saint louis univercity": "Saint Louis University",
    "saint louis univesity": "Saint Louis University",
    "saint louis univesrity": "Saint Louis University",
    "saint louis university saint louis missouri": "Saint Louis University",
    "st louis university": "Saint Louis University",
    "st. louis university": "Saint Louis University",
    "st. louis University": "Saint Louis University",
    "slu": "Saint Louis University",
    "saintlouisuniversity": "Saint Louis University",
    "saint louis universty": "Saint Louis University",

    # Illinois Institute of Technology
    "illinois institute of technology": "Illinois Institute of Technology",
    "illinois institute of technology chicago": "Illinois Institute of Technology",
    "illinois institute of techonology": "Illinois Institute of Technology",
    "illinois institute of technnology": "Illinois Institute of Technology",
    "illinois intitute of technology": "Illinois Institute of Technology",
    "illinois institution of technology": "Illinois Institute of Technology",
    "illinos institute of technology": "Illinois Institute of Technology",
    "ill": "Illinois Institute of Technology",
    "illinois tech university": "Illinois Institute of Technology",
    "illinois institute of technology": "Illinois Institute of Technology",

    # JNTU family
    "jntuh": "Jawaharlal Nehru Technological University Hyderabad",
    "jntuhceh": "Jawaharlal Nehru Technological University Hyderabad",
    "jntu": "Jawaharlal Nehru Technological University Hyderabad",
    "jntuk": "Jawaharlal Nehru Technological University Kakinada",
    "jntuacek": "Jawaharlal Nehru Technological University Kakinada",
    "jntuk-university college of engineering vizianagaram": "Jawaharlal Nehru Technological University Kakinada",

    # SRM family
    "srm institute of science and technology": "SRM Institute of Science and Technology",
    "srm ist": "SRM Institute of Science and Technology",
    "srm university": "SRM Institute of Science and Technology",
    "srm university ktr": "SRM Institute of Science and Technology",
    "srm university ap": "SRM Institute of Science and Technology",
    "srm university ramapuram chennai": "SRM Institute of Science and Technology",
    "srmist ramapuram": "SRM Institute of Science and Technology",

    # VIT family
    "vellore institute of technology": "Vellore Institute of Technology",
    "vit university": "Vellore Institute of Technology",
    "vit university vellore": "Vellore Institute of Technology",
    "vit ap university": "Vellore Institute of Technology - Andhra Pradesh",
    "vit bhopal university": "Vellore Institute of Technology Bhopal",
    "vit chennai": "Vellore Institute of Technology Chennai",

    # University of Ibadan
    "university of ibadan": "University of Ibadan",
    "university of ibadan ibadan nigeria.": "University of Ibadan",

    # University of Port Harcourt
    "university of port harcourt": "University of Port Harcourt",
    "university of portharcourt": "University of Port Harcourt",
    "university of port-harcourt": "University of Port Harcourt",

    # Kwame Nkrumah University of Science and Technology
    "kwame nkrumah university of science and technology": "Kwame Nkrumah University of Science and Technology",
    "kwame nkrumah university of science and technology knust": "Kwame Nkrumah University of Science and Technology",
    "knust": "Kwame Nkrumah University of Science and Technology",

    # Obafemi Awolowo University
    "obafemi awolowo university": "Obafemi Awolowo University",
    "oau ile ife": "Obafemi Awolowo University",

    # University of Ilorin
    "university of ilorin": "University of Ilorin",
    "university of ilorin ilorin kwara state": "University of Ilorin",
    "unilorin": "University of Ilorin",

    # University of Benin
    "university of benin": "University of Benin",
    "uniben": "University of Benin",

    # Federal University of Technology Owerri
    "federal university of technology owerri": "Federal University of Technology Owerri",
    "federal university of tecnology owerri": "Federal University of Technology Owerri",

    # Vignan family
    "vignan's lara institute of technology and science": "Vignan's Lara Institute of Technology and Science",
    "vignan's lara institute of technology & science": "Vignan's Lara Institute of Technology and Science",
    "vignan institute of technology and science": "Vignan Institute of Technology and Science",

    # Webster University
    "webster university": "Webster University",

    # Chandigarh University
    "chandigarh university": "Chandigarh University",

    # CMR family
    "cmr engineering college": "CMR Engineering College",
    "cmr college of engineering and technology": "CMR College of Engineering and Technology",
    "cmr institute of technology": "CMR Institute of Technology",
    "cmr technical campus": "CMR Technical Campus",

    # Anurag University / Group
    "anurag university": "Anurag University",
    "anurag group of institutions": "Anurag University",

    # University of Ibadan
    "university of ibadan": "University of Ibadan",
    
    # JNTU
    "jntuh": "Jawaharlal Nehru Technological University Hyderabad",
    "jntu": "Jawaharlal Nehru Technological University Hyderabad",
    "jntuk": "Jawaharlal Nehru Technological University Kakinada",
    
    # Vishnu Institute
    "vishnu institute of technology": "Vishnu Institute of Technology",
    
    # University of Port Harcourt
    "university of port harcourt": "University of Port Harcourt",
    "university of portharcourt": "University of Port Harcourt",
    
    # Chandigarh University
    "chandigarh university": "Chandigarh University",
    
    # University of Benin
    "university of benin": "University of Benin",
    "uniben": "University of Benin",
    
    # CMR
    "cmr engineering college": "CMR Engineering College",
    "cmr college of engineering and technology": "CMR Engineering College",
    "cmr institute of technology": "CMR Engineering College",
    
    # University of Ilorin
    "university of ilorin": "University of Ilorin",
    
    # Vignan's Lara
    "vignan's lara institute of technology and science": "Vignan's Lara Institute of Technology and Science",
    "vignans lara institute of technology and science": "Vignan's Lara Institute of Technology and Science",
    
    # Accra Technical University
    "accra technical university": "Accra Technical University",
    
    # GITAM
    "gitam university": "GITAM University",
    "gitam": "GITAM University",
    
    # Obafemi Awolowo
    "obafemi awolowo university": "Obafemi Awolowo University",
    "oau ile ife": "Obafemi Awolowo University",
    
    # Dhanekula
    "dhanekula institute of engineering and technology": "Dhanekula Institute of Engineering and Technology",
    
    # Pune University / SPPU
    "pune university": "Savitribai Phule Pune University",
    "savitribai phule pune university": "Savitribai Phule Pune University",
    
    # University of Uyo
    "university of uyo": "University of Uyo",
    
    # Vemana
    "vemana institute of technology": "Vemana Institute of Technology",
    
    # MIT-WPU
    "mit-wpu": "MIT World Peace University",
    "mit world peace university": "MIT World Peace University",
    
    # FUTO
    "federal university of technology owerri": "Federal University of Technology Owerri",
    
    # NOUN
    "national open university of nigeria": "National Open University of Nigeria",}

def normalize_institution(name):
    if pd.isna(name):
        return name

    original = str(name)
    lower = original.strip().lower()
    
    # Remove common noise
    lower = re.sub(r'\s*\(.*?\)\s*', '', lower)  
    lower = re.sub(r'\s*-\s*', ' ', lower)
    lower = re.sub(r'\s+', ' ', lower)
    lower = re.sub(r'[.,;]$', '', lower)  
    
    # Direct mapping
    if lower in norm_to_canonical:
        return norm_to_canonical[lower]
    
    # Partial/fuzzy fallback for remaining big groups  
    if 'saint louis' in lower or 'st louis' in lower or 'slu' in lower:
        return "Saint Louis University"
    if 'illinois institute of technology' in lower or 'iit chicago' in lower:
        return "Illinois Institute of Technology"
    if 'jntu' in lower:
        if 'hyderabad' in lower:
            return "Jawaharlal Nehru Technological University Hyderabad"
        if 'kakinada' in lower:
            return "Jawaharlal Nehru Technological University Kakinada"
    if 'srm' in lower:
        return "SRM Institute of Science and Technology"
    if 'vit' in lower or 'vellore institute' in lower:
        return "Vellore Institute of Technology"
    if 'kwame nkrumah' in lower or 'knust' in lower:
        return "Kwame Nkrumah University of Science and Technology"
    if 'university of ghana' in lower or 'legon' in lower:
        return "University of Ghana"
    if 'university of port harcourt' in lower or 'uniport' in lower:
        return "University of Port Harcourt"
    if 'university of benin' in lower or 'uniben' in lower:
        return "University of Benin"
    if 'university of ilorin' in lower or 'unilorin' in lower:
        return "University of Ilorin"
    if 'university of ibadan' in lower:
        return "University of Ibadan"
    if 'federal university of technology owerri' in lower or 'futo' in lower:
        return "Federal University of Technology Owerri"
        
     # High schools / junk / incomplete
    if any(word in lower for word in ['high school', 'senior high', 'secondary school', 'academy', 'junior college', 'gap year', 'homeschooled']):
        return "High School / Pre-University"
    if lower in ['nan', '', 'nil', 'n/a', 'not applicable', 'applying', 'employment']:
        return "Unknown / Not Specified"
    
    # Final cleanup: title case
    cleaned = original.strip()
    if cleaned == "" or cleaned.lower() in ['nan', 'nil']:
        return "Unknown"
    return cleaned.title()

df['Institution Cleaned'] = df['Institution Name'].apply(normalize_institution)

print("Top 30 Institutions after advanced cleaning:")
print(df['Institution Cleaned'].value_counts().head(30))

print("\nNumber of unique institutions now:", df['Institution Cleaned'].nunique())
print("\nInstitutions with >10 applicants:")
print(df['Institution Cleaned'].value_counts()[df['Institution Cleaned'].value_counts() > 10])

Top 30 Institutions after advanced cleaning:
Institution Cleaned
Saint Louis University                                 4516
High School / Pre-University                            181
Illinois Institute of Technology                        174
Webster University                                       72
SRM Institute of Science and Technology                  60
Kwame Nkrumah University of Science and Technology       55
Vellore Institute of Technology                          37
University of Port Harcourt                              31
CMR Engineering College                                  30
University of Ibadan                                     28
Anurag University                                        27
University of Ilorin                                     26
Vishnu Institute of Technology                           25
Jawaharlal Nehru Technological University Hyderabad      25
Chandigarh University                                    24
University of Benin                

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8263 non-null   datetime64[ns]
 1   Opportunity Id           8558 non-null   object        
 2   Opportunity Name         8558 non-null   object        
 3   Opportunity Category     8558 non-null   object        
 4   Opportunity End Date     7296 non-null   datetime64[ns]
 5   First Name               8558 non-null   object        
 6   Date of Birth            8558 non-null   datetime64[ns]
 7   Gender                   8558 non-null   object        
 8   Country                  8558 non-null   object        
 9   Institution Name         8553 non-null   object        
 10  Current/Intended Major   8553 non-null   object        
 11  Entry created at         8558 non-null   datetime64[ns]
 12  Status Description       8558 non-

In [6]:
print("Null counts per column:")
print(df.isna().sum())

print("\nPercentage of nulls:")
print((df.isna().mean() * 100).round(2))

print("\nSample rows with null Opportunity Start Date:")
print(df[df['Opportunity Start Date'].isna()].head()[['Opportunity Name', 'Opportunity Start Date', 'Apply Date']])

Null counts per column:
Learner SignUp DateTime     295
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date       1262
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              5
Current/Intended Major        5
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                  307
Opportunity Start Date     4637
Institution Cleaned           5
dtype: int64

Percentage of nulls:
Learner SignUp DateTime     3.45
Opportunity Id              0.00
Opportunity Name            0.00
Opportunity Category        0.00
Opportunity End Date       14.75
First Name                  0.00
Date of Birth               0.00
Gender                      0.00
Country                     0.00
Institution Name            0.06
Current/Intended Major      0.06
Entry created at            0.00
S

In [7]:
df[df.duplicated()]

,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date,Institution Cleaned


In [8]:
print("Missing values summary:")
print(df.isna().sum())

df['Apply YearMonth'] = df['Apply Date'].dt.strftime('%Y-%m')  # e.g., 2023-06
df['Apply Year'] = df['Apply Date'].dt.year
df['Apply Month'] = df['Apply Date'].dt.month
df['Signup to Apply Days'] = (df['Apply Date'] - df['Learner SignUp DateTime']).dt.days

df['Age at Apply'] = (df['Apply Date'].dt.year - df['Date of Birth'].dt.year)

threshold = 20
inst_counts = df['Institution Cleaned'].value_counts()
df['Institution Group'] = df['Institution Cleaned'].apply(
    lambda x: x if inst_counts.get(x, 0) >= threshold else "Other Institutions"
)

df['Country_Institution'] = df['Country'] + " - " + df['Institution Group']

df['Status Simple'] = df['Status Description'].replace({
    'Started': 'Completed/Started',
    'Team Allocated': 'In Progress',
    'Registered': 'Registered',
}).fillna('Other')

df.to_csv('cleaned_slu_opportunity_data.csv', index=False)

Missing values summary:
Learner SignUp DateTime     295
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date       1262
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              5
Current/Intended Major        5
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                  307
Opportunity Start Date     4637
Institution Cleaned           5
dtype: int64


In [9]:
clean_df = pd.read_csv("/kaggle/working/cleaned_slu_opportunity_data.csv")
clean_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,...,Opportunity Start Date,Institution Cleaned,Apply YearMonth,Apply Year,Apply Month,Signup to Apply Days,Age at Apply,Institution Group,Country_Institution,Status Simple
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Faria,2001-12-01,Female,Pakistan,Nwihs,...,2022-03-11 18:30:39,Nwihs,2023-06,2023.0,6.0,0.0,22.0,Other Institutions,Pakistan - Other Institutions,Completed/Started
1,2023-01-05 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Poojitha,2000-08-16,Female,India,SAINT LOUIS,...,2022-03-11 18:30:39,Saint Louis University,2023-01,2023.0,1.0,0.0,23.0,Saint Louis University,India - Saint Louis University,Completed/Started
2,2023-09-04 20:35:08,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Emmanuel,2002-01-27,Male,United States,Illinois Institute of Technology,...,2022-03-11 18:30:39,Illinois Institute of Technology,NaN,NaN,NaN,NaN,NaN,Illinois Institute of Technology,United States - Illinois Institute of Technology,Completed/Started
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Amrutha Varshini,1999-01-11,Female,United States,Saint Louis University,...,2022-03-11 18:30:39,Saint Louis University,2023-09,2023.0,9.0,12.0,24.0,Saint Louis University,United States - Saint Louis University,In Progress
4,2023-06-01 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Vinay Varshith,2000-04-19,Male,United States,Saint Louis University,...,2022-03-11 18:30:39,Saint Louis University,2023-06,2023.0,6.0,0.0,23.0,Saint Louis University,United States - Saint Louis University,Completed/Started
